# RECOIL Intermodal Data Explorer (Voilà)

This web app lets you explore intermodal freight nodes, edges (H/R/W), and demand on-demand without pre-loading heavy datasets.

**How to use:**
- Enter a city name (or leave blank for all)
- Select a mode (Highway, Railway, or Waterway)
- Click Load Nodes or Load Edges to fetch and display data

Data is fetched live from: https://recoil.ise.utk.edu/data/Parsed_Data/

In [ ]:
# Imports and configuration

import os
import pickle
import requests
import pandas as pd
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

# Base URL for remote data
DEFAULT_BASE_URL = 'https://recoil.ise.utk.edu/data/Parsed_Data/'
BASE_URL = os.getenv('BASE_URL', DEFAULT_BASE_URL)

# Helper to load pickles from URL
def load_pickle_from_url(url: str):
    try:
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()
        return pickle.loads(resp.content)
    except Exception as e:
        raise RuntimeError(f"Failed to load pickle from {url}: {e}")

# Helper to create GeoDataFrame from edges dict
def create_geoframe_from_edges(edges: dict, sample: int | None = None):
    from shapely import LineString
    rows = []
    keys = list(edges.keys())
    if sample is not None:
        keys = keys[:sample]
    for (i, j) in keys:
        rec = edges[(i, j)]
        path = LineString([(lon, lat) for (lat, lon) in rec['path']])
        rows.append([i, j, rec['i_lat'], rec['i_lon'], rec['j_lat'], rec['j_lon'], rec['mode'], rec['distance'], path])
    gdf = gpd.GeoDataFrame(rows, columns=['i','j','i_lat','i_lon','j_lat','j_lon','mode','distance','geometry'], geometry='geometry', crs='EPSG:4326')
    return gdf

In [ ]:
# Interactive panel

city_input = widgets.Text(value='', description='City:', placeholder='City substring (leave blank for all)')
mode_dropdown = widgets.Dropdown(options=['H','R','W'], value='W', description='Mode:')
load_nodes_btn = widgets.Button(description='Load Nodes', button_style='info')
load_edges_btn = widgets.Button(description='Load Edges (sample)', button_style='primary')
load_demand_btn = widgets.Button(description='Load Demand (sample)', button_style='success')
out = widgets.Output()

def on_load_nodes(_):
    with out:
        out.clear_output()
        print('Loading nodes CSV...')
        url_N = f"{BASE_URL}intermodal-217.csv"
        try:
            df_nodes = pd.read_csv(url_N)
            if city_input.value:
                df_nodes = df_nodes[df_nodes['city'].str.contains(city_input.value, case=False, na=False)]
            display(HTML(f'<h4>Nodes (showing up to 20)</h4>'))
            display(HTML(df_nodes.head(20).to_html(index=False)))
            print(f"Total rows: {len(df_nodes)}")
        except Exception as e:
            print(f"Error loading nodes: {e}")

def on_load_edges(_):
    with out:
        out.clear_output()
        mode = mode_dropdown.value
        mode_map = {'H': 'Highway', 'R': 'Railway', 'W': 'Waterway'}
        url_map = {
            'H': f"{BASE_URL}H-adj.pickle",
            'R': f"{BASE_URL}R-adj.pickle",
            'W': f"{BASE_URL}W-adj.pickle",
        }
        url = url_map[mode]
        print(f"Loading {mode_map[mode]} edges from remote source...")
        try:
            edges = load_pickle_from_url(url)
            print(f"Total edges: {len(edges)}")
            gdf = create_geoframe_from_edges(edges, sample=15)
            display(HTML(f'<h4>{mode_map[mode]} Edges (sample of 15)</h4>'))
            display(gdf[['i','j','mode','distance']].head(15))
            try:
                ax = gdf.to_crs(3857).plot(figsize=(8,5), linewidth=1.5, color='blue')
                ax.set_title(f'{mode_map[mode]} Network (sample)')
                ax.set_axis_off()
                plt.tight_layout()
                plt.show()
            except Exception as plot_e:
                print(f"Plot skipped: {plot_e}")
        except Exception as e:
            print(f"Error loading edges: {e}")

def on_load_demand(_):
    with out:
        out.clear_output()
        url_D = f"{BASE_URL}demand.pickle"
        print(f"Loading demand data from remote source...")
        try:
            demand = load_pickle_from_url(url_D)
            print(f"Total demand pairs: {len(demand)}")
            sample_keys = list(demand.keys())[:10]
            rows = []
            for (i, j) in sample_keys:
                rec = demand[(i, j)]
                rows.append([i, j, rec.get('tons_2025', 0), rec.get('tons_2030', 0), rec.get('tons_2050', 0)])
            df = pd.DataFrame(rows, columns=['Origin', 'Dest', 'Tons 2025', 'Tons 2030', 'Tons 2050'])
            display(HTML(f'<h4>Demand (sample of 10 OD pairs)</h4>'))
            display(HTML(df.to_html(index=False)))
        except Exception as e:
            print(f"Error loading demand: {e}")

load_nodes_btn.on_click(on_load_nodes)
load_edges_btn.on_click(on_load_edges)
load_demand_btn.on_click(on_load_demand)

ui = widgets.VBox([
    widgets.HTML('<h3>Explore Intermodal Data (on demand)</h3>'),
    widgets.HBox([city_input, mode_dropdown]),
    widgets.HBox([load_nodes_btn, load_edges_btn, load_demand_btn]),
    out
])
display(ui)

---

## About the data

- **Nodes**: 217 intermodal locations (Highway: 301–413, Railway: 101–156, Waterway: 201–248)
- **Edges**: mode-specific adjacency with distance and path geometry
- **Demand**: directed OD pairs with tonnage projections (2025–2050)

For detailed exploration and the full notebook, see [`usage.ipynb`](usage.ipynb) in the repo.